In [ ]:
!pip install -q streamlit
!npm install localtunnel


In [ ]:
#copypasta and create 

%%writefile app.py

### BACKEND

from PIL import Image
from tqdm.auto import tqdm
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler, LMSDiscreteScheduler, StableDiffusionPipeline, UniPCMultistepScheduler
from torchvision import transforms

torch_device = "cuda" if torch.cuda.is_available() else ("mps" if torch.mps.is_available() else "cpu")

torch_dtype = torch.float16 if torch_device in ["cuda", "mps"] else torch.float32

pipe = StableDiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", 
    torch_dtype=torch_dtype, 
    use_safetensors=True, 
    safety_checker = None).to(torch_device)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

pipe.enable_model_cpu_offload()

def StableDiffusion(uncond_embeddings, text_embeddings, height, width, num_inference_steps, guidance_scale, seed):
    batch_size=1

    generator = None

    if seed:
        generator=torch.manual_seed(seed)
    
    output = pipe(
        prompt = text_embeddings,
        negative_prompt = uncond_embeddings,
        height = height,
        width = width,
        num_inference_steps = num_inference_steps,
        guidance_scale = guidance_scale,
        generator = generator
    ).images[0]
    return output



### FRONTEND
import streamlit as st
import random
from PIL import Image
import io

st.title("Image Generator App")

# Input fields
prompt = st.text_input("Prompt", value="").split(',')
negative_prompt = st.text_input("Negative Prompt", value="").split(',')
height = st.number_input("Height", min_value=1, max_value=1024, value=512)
width = st.number_input("Width", min_value=1, max_value=1024, value=512)
num_inference_steps = st.number_input("Number of Inference Steps", min_value=1, max_value=100, value=50)
guidance_scale = st.slider("Guidance Scale", min_value=1.0, max_value=50.0, value=7.5)
seed = st.number_input("Seed", min_value=0, value=42)

# Generate button
if st.button("Generate Image"):
    # Generate the image
    generated_image = StableDiffusion(uncond_embeddings=negative_prompt, 
                                      text_embeddings=prompt, 
                                      height=height, 
                                      width=width, 
                                      num_inference_steps=num_inference_steps, 
                                      guidance_scale=guidance_scale, 
                                      seed=seed, 
                                      image=False)

    # Display the generated image
    st.image(generated_image, caption="Generated Image", use_column_width=True)

# Note about the placeholder function
st.markdown("""
I made this in like 5 minutes don't look at me.
""")

# HOW TO RUN THE GUI:
Run this next cell, and then click the link generated. When it asks for a password, copy and paste the printed IP address and click "enter". IT should now work.

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com
